<a href="https://colab.research.google.com/github/stephkariuki19/final-project-colabs/blob/main/projectIterOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
DISEASES:
THE DISEASES:
Pneumonia
Consolidation
Infiltration
Pneumothorax
Asthma
Emphysema
Fibrosis
Covid- 19
 Effusion
Atelectasis
Pleural_thickening
Cardiomegaly
Nodule Mass
TB
'''

In [ ]:
!pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

In [ ]:
pip install langchain-community

In [3]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.mayoclinic.org/diseases-conditions/enlarged-heart/symptoms-causes/syc-20355436"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()




Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.35it/s]


In [5]:
pip install html2text

In [ ]:
#function to output results of webpage only
from langchain_community.document_transformers import Html2TextTransformer

# Define the topics of interest
topics = ['symptoms', 'when to see a doctor', 'causes', 'risk factors', 'complications', 'prevention', 'vaccination']

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Dictionary to store content for each topic
topic_content = {topic: [] for topic in topics}

# Iterate over transformed documents
for doc in docs_transformed:
    # Iterate over each topic
    for topic in topics:
        # Find the index of the topic in the page content
        index = doc.page_content.lower().find(topic)
        if index != -1:
            # Find the end index of the topic's section
            next_topic_index = len(doc.page_content)
            for next_topic in topics:
                next_index = doc.page_content.lower().find(next_topic)
                if next_index != -1 and next_index > index:
                    next_topic_index = min(next_topic_index, next_index)
            # Extract content for the topic
            topic_content[topic].append(doc.page_content[index:next_topic_index])

# Print content for each topic
for topic, content_list in topic_content.items():
    print(f"Content for topic '{topic}':")
    for content in content_list:
        print(content)
        print("---------------------")


In [6]:
#function to output results of webpage and save to my Google drive
from google.colab import drive
from langchain_community.document_transformers import Html2TextTransformer

# Mount Google Drive
drive.mount('/content/drive',force_remount=True)

# Define the topics of interest
topics = ['symptoms', 'when to see a doctor', 'causes', 'risk factors', 'complications', 'prevention', 'vaccination']

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Dictionary to store content for each topic
topic_content = {topic: [] for topic in topics}

# Iterate over transformed documents
for doc in docs_transformed:
    # Iterate over each topic
    for topic in topics:
        # Find the index of the topic in the page content
        index = doc.page_content.lower().find(topic)
        if index != -1:
            # Find the end index of the topic's section
            next_topic_index = len(doc.page_content)
            for next_topic in topics:
                next_index = doc.page_content.lower().find(next_topic)
                if next_index != -1 and next_index > index:
                    next_topic_index = min(next_topic_index, next_index)
            # Extract content for the topic
            topic_content[topic].append(doc.page_content[index:next_topic_index])

# Write content for each topic to pneumonia.txt in Google Drive
file_path = '/content/drive/My Drive/Cardiomegaly.txt'
with open(file_path, 'w', encoding='utf-8') as file:
    for topic, content_list in topic_content.items():
        file.write(f"Content for topic '{topic}':\n")
        for content in content_list:
            file.write(content + "\n")
            file.write("---------------------\n")

print(f"Content written to {file_path} successfully.")


Mounted at /content/drive
Content written to /content/drive/My Drive/Cardiomegaly.txt successfully.
